## google images web scrapper

Little project to scrap images from google. a list of keywords can be specified as a search parameter.

this article proved to be usefull as scraping images from google requires some more unconventional methods as they have put in place lots of protection against web scraping:
https://medium.com/france-school-of-ai/scraper-un-dataset-depuis-google-images-342f670b9bad

In [1]:
import selenium
import selenium.webdriver
import time
import numpy as np



In [2]:
driver = selenium.webdriver.Chrome("/mnt/c/Program Files (x86)/Google/Chrome/Application/chromedriver.exe")

In [10]:
start_url = "https://www.google.com/imghp"
js_image_links_selector = "return Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);"
js_scrolldown = "window.scrollTo(0, document.body.scrollHeight);"

driver.get(start_url)


input_box = driver.find_element_by_class_name("gLFyf")
input_box.send_keys("shipibo" + "\n")

#we scrolldown once so that we can get more picctures
driver.execute_script(js_scrolldown)
time.sleep(5)
links= driver.execute_script(js_image_links_selector)



print(len(links))

200


# 1) link collection
Instead of going through every single image on google, right clicking and selecting "open image ina  new tab" which i'm pretty sur is not even possible even using selenium we will instead run a small javascript function that will return a list of links that will have the same effect as doing the tedious operation mentioned above. 

This will avoid us from eventually getting our IP address blocked by google after frantiaclly looking at every picture that they have and downloading them like a madman. This also means that we have no need for using a proxy. This wouldve been necessary had we been scraping images from a single provider; but, it is most likely that the pictures that we will be downloading will come from various content managers (cdn) which will not find our downloading activity very suspicious.

## the function

This function will therfore have a list of terms to search for, all relating to the subject that we wish to scrape images from. For every search term, we will first go down in order to have google load an extra 100 pictures for us (we could repeat this to collect up to 700 images but they will not all be relevant by that stage). after this part, we will execute a small javascript function (idk how it works) that will return all of the proper links from which we can directly download the images from.

In [3]:
def collect_links(search_terms,driver):
    #google images home page
    start_url = "https://www.google.com/imghp"
    #javascript functions that we will use to gather the links
    js_image_links_selector = "return Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);"
    js_scrolldown = "window.scrollTo(0, document.body.scrollHeight);"
    
    image_link_list = []
    
    for term in search_terms:
    
        driver.get(start_url)
        #find search bar
        input_box = driver.find_element_by_class_name("gLFyf")
        input_box.send_keys(term + "\n")
        
        #we scrolldown once so that we can get more picctures
        driver.execute_script(js_scrolldown)
        #sleep a random amount of time, we need enough to let google load the rest of the pictures
        time.sleep(np.random.randint(2,5))
        driver.execute_script(js_scrolldown) #repeat to get 300 links per term searched
        time.sleep(np.random.randint(2,5))
        
        term_links = driver.execute_script(js_image_links_selector)
        image_link_list.append(term_links)
        
        
        #sleep again for a random amount of time
        time.sleep(np.random.randint(1,4))
        
    return image_link_list


terms_list = ["shipibo art", "shipibo patterns","shipibo tapestry","shipibo textiles"]
img_links = collect_links(terms_list,driver)

In [4]:

for links in img_links:
    print(len(links))

300
300
300
300


# 2) downloading the images
Now that we have gathered all of the necessary links, we can move on to downloading the images. In order to do this, we will us the fast.ai library that has a module allowing us to download image files with ease

https://www.geeksforgeeks.org/reading-writing-text-files-python/: working with files

In [5]:
#make a quick .txt file containing the links, we will need it to give to the fastai lib


test_links = []
for links in img_links:
    test_links.extend(links)

test_links_content = "\n".join(test_links)

#open in write-only mode to truncate the existing links
test_file = open("links.txt","w")
test_file.write(test_links_content)
test_file.close()

In [7]:
from pathlib import Path
import os

from fastai.vision import download_images

dest_path = "../../assets/assetsv2/"
links = "links.txt"


download_images(links,dest_path)

